In [1]:
%pip install pandas
%pip install mlxtend


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
base = pd.read_csv('../_Base/Supermercado.csv', sep=';', encoding='cp1252', header=None)
base

,0,1,2,3,4,5,6
0,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
1,Não,Sim,Não,Sim,Sim,Não,Não
2,Sim,Não,Sim,Sim,Sim,Não,Não
3,Não,Sim,Não,Sim,Sim,Não,Não
4,Sim,Sim,Não,Sim,Sim,Não,Não
5,Não,Não,Sim,Não,Não,Não,Não
6,Não,Não,Não,Não,Sim,Não,Não
7,Não,Não,Não,Sim,Não,Não,Não
8,Não,Não,Não,Não,Não,Não,Sim
9,Não,Não,Não,Não,Não,Sim,Sim


In [4]:
for column in base:
    base[column]=base[column].fillna('~' + str(column))
    
base



,0,1,2,3,4,5,6
0,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
1,Não,Sim,Não,Sim,Sim,Não,Não
2,Sim,Não,Sim,Sim,Sim,Não,Não
3,Não,Sim,Não,Sim,Sim,Não,Não
4,Sim,Sim,Não,Sim,Sim,Não,Não
5,Não,Não,Sim,Não,Não,Não,Não
6,Não,Não,Não,Não,Sim,Não,Não
7,Não,Não,Não,Sim,Não,Não,Não
8,Não,Não,Não,Não,Não,Não,Sim
9,Não,Não,Não,Não,Não,Sim,Sim


In [5]:
transactions = []
for i in range(0, len(base)):
    transactions.append([str(base.values[i,j]) for j in range(0, len(base.columns))])

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

In [6]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.090909,(Arroz),1
1,0.090909,(Café),1
2,0.090909,(Cerveja),1
3,0.090909,(Feijão),1
4,0.090909,(Leite),1
...,...,...,...
125,0.090909,"(Pão, Manteiga, Arroz, Cerveja, Leite, Café)",6
126,0.090909,"(Pão, Manteiga, Arroz, Feijão, Leite, Café)",6
127,0.090909,"(Pão, Manteiga, Arroz, Feijão, Cerveja, Leite)",6
128,0.090909,"(Pão, Manteiga, Feijão, Cerveja, Leite, Café)",6


In [7]:
print(frequent_itemsets[['itemsets', 'support']])


                                              itemsets   support
0                                              (Arroz)  0.090909
1                                               (Café)  0.090909
2                                            (Cerveja)  0.090909
3                                             (Feijão)  0.090909
4                                              (Leite)  0.090909
..                                                 ...       ...
125       (Pão, Manteiga, Arroz, Cerveja, Leite, Café)  0.090909
126        (Pão, Manteiga, Arroz, Feijão, Leite, Café)  0.090909
127     (Pão, Manteiga, Arroz, Feijão, Cerveja, Leite)  0.090909
128      (Pão, Manteiga, Feijão, Cerveja, Leite, Café)  0.090909
129  (Pão, Manteiga, Arroz, Feijão, Cerveja, Leite,...  0.090909

[130 rows x 2 columns]


In [8]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)
rules['antecedent'] = rules['antecedents'].apply(lambda x: list(x)[0] if len(x) > 0 else 'None')
rules['consequent'] = rules['consequents'].apply(lambda x: list(x)[0] if len(x) > 0 else 'None')
rules = rules[['antecedent', 'consequent', 'support', 'confidence', 'lift']]
rules


,antecedent,consequent,support,confidence,lift
0,Arroz,Café,0.090909,1.0,11.0
1,Café,Arroz,0.090909,1.0,11.0
2,Arroz,Cerveja,0.090909,1.0,11.0
3,Cerveja,Arroz,0.090909,1.0,11.0
4,Arroz,Feijão,0.090909,1.0,11.0
...,...,...,...,...,...
1929,Arroz,Pão,0.090909,1.0,11.0
1930,Feijão,Pão,0.090909,1.0,11.0
1931,Cerveja,Pão,0.090909,1.0,11.0
1932,Leite,Pão,0.090909,1.0,11.0
